# 팔당댐 수위 예측 baseline

### 필요한 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

import warnings; warnings.filterwarnings(action='ignore') # 경고 메시지 무시

'''#GPU 자원이 부족한 경우 아래 코드를 이용
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)'''

### 데이터 전처리

In [7]:
w_list = sorted(glob("C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data/*.csv"))
w_list

['C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2012.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2013.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2014.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2015.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2016.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2017.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2018.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2019.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2020.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2021.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2022.csv']

In [8]:
pd.read_csv(w_list[0]).shape

(26496, 15)

In [9]:
pd.read_csv(w_list[0]).head(4)

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [13]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
    
    tmp = pd.read_csv(i)
    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)
    
    # 431은 3일 간격을 의미하며, 432는 예측하고자 하는 시간을 의미
    
    for j in tqdm(range(len(tmp)-432)):
        train_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                       "tototf", "tide_level",
                                                       "fw_1018662", "fw_1018680",
                                                       "fw_1018683", "fw_1019630"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:38<00:00, 677.93it/s]


In [14]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 10)
(260640, 4)


### 모델링 및 모델 학습

In [21]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(4, activation = 'relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer,loss='mse', metrics=['mae'])

In [22]:
model.fit(train_data, train_label, epochs=10, batch_size=128)

Epoch 1/10
2037/2037 [==============================] - 4508s 2s/step - loss: 23242.5645 - mae: 104.6861
Epoch 2/10
2037/2037 [==============================] - 4370s 2s/step - loss: 772.0200 - mae: 10.0065
Epoch 3/10
2037/2037 [==============================] - 4278s 2s/step - loss: 436.4344 - mae: 8.1622
Epoch 4/10
2037/2037 [==============================] - 4197s 2s/step - loss: 365.2337 - mae: 7.8262
Epoch 5/10
2037/2037 [==============================] - 4211s 2s/step - loss: 304.0364 - mae: 7.5309
Epoch 6/10
2037/2037 [==============================] - 4161s 2s/step - loss: 278.2153 - mae: 7.6156
Epoch 7/10
2037/2037 [==============================] - 4146s 2s/step - loss: 252.8148 - mae: 7.4774
Epoch 8/10
2037/2037 [==============================] - 4158s 2s/step - loss: 279.0431 - mae: 7.9246
Epoch 9/10
2037/2037 [==============================] - 4171s 2s/step - loss: 274.0914 - mae: 8.0982
Epoch 10/10
2037/2037 [==============================] - 4200s 2s/step - loss: 275.666

### 추론 데이터셋 만들기

In [23]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)
# 이전값을 사용
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
    
#tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100

# 431은 3일 간격을 의미하며, 432는 예측하고자 하는 시간을 의미
    
for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                    "tototf", "tide_level",
                                                    "fw_1018662", "fw_1018680",
                                                    "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:08<00:00, 789.75it/s]


In [24]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 10)
(6912, 4)


### 제출 파일 만들기

In [26]:
pred = model.predict(test_data)

pred = pd.DataFrame(pred)

sample_submission = pd.read_csv("C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

216/216 [==============================] - 32s 150ms/step


In [29]:
sample_submission

,ymdhm,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,2022-06-01 00:00,288.294586,279.388550,277.245331,274.106812
1,2022-06-01 00:10,280.480438,270.659668,267.147064,260.063507
2,2022-06-01 00:20,284.233276,273.763519,270.000305,261.181610
3,2022-06-01 00:30,292.709930,281.110992,277.062744,265.448303
4,2022-06-01 00:40,283.942047,273.510498,269.756592,261.032043
...,...,...,...,...,...
6907,2022-07-18 23:10,304.334290,298.403717,299.291382,306.808441
6908,2022-07-18 23:20,302.822571,297.095612,298.035553,306.057983
6909,2022-07-18 23:30,306.740906,300.492249,301.300659,308.031586
6910,2022-07-18 23:40,326.638214,317.740356,317.880829,318.053589


In [62]:
sample_submission.to_csv("baseline.csv", index = False)